# LangChain LCEL Application Implementation

**Author**: Sergio Masa Avis  
**Date**: October 2025

Comprehensive implementation of a LLM application using LangChain and LCEL (LangChain Expression Language). Demonstrates chain composition, prompt templating, and output parsing patterns for AI applications.

---

## Table of Contents

1. [LangChain and LCEL Overview](#langchain-and-lcel-overview)
2. [Architecture and Components](#architecture-and-components)
3. [Implementation](#implementation)
4. [Trip Planner Application](#trip-planner-application)
5. [Advanced Features](#advanced-features)
6. [References](#references)

---

## 1. LangChain and LCEL Overview

**LangChain** is a comprehensive framework for developing applications with large language models (LLMs), providing abstractions for prompt management, chain composition, and integration with external data sources and APIs.

**LCEL (LangChain Expression Language)** is a declarative syntax for composing LangChain components into execution chains using the pipe operator (`|`).

### Core Concepts

**Chain Composition**: Sequential component execution through standardized interfaces

```python
chain = prompt | model | parser
```

**Runnable Interface**: Common protocol for all LangChain components enabling interoperability

**Template-Based Prompts**: Dynamic prompt generation with variable substitution

### LCEL Benefits

| Feature | Benefit | Implementation |
|---------|---------|----------------|
| **Streaming** | Real-time response generation | Automatic chunked output |
| **Async Support** | Concurrent execution | Native async/await compatibility |
| **Batch Processing** | Multiple input handling | Vectorized operations |
| **Parallelization** | Performance optimization | Automatic dependency resolution |
| **Logging** | Observability | Built-in execution tracing |
| **Fallbacks** | Error resilience | Alternative execution paths |

### Problem Statement

Traditional LLM applications face several challenges:
- **Prompt Management**: Hard-coded prompts, difficult versioning
- **Component Integration**: Manual wiring between different services
- **Error Handling**: Limited resilience and retry mechanisms
- **Observability**: Poor visibility into execution flow
- **Scalability**: No built-in support for async or batch processing

LangChain + LCEL solutions:
- Standardized component interfaces through Runnable protocol
- Declarative chain composition with automatic optimization
- Built-in error handling, retries, and fallback mechanisms
- Comprehensive logging and tracing capabilities
- Native support for streaming, async, and batch operations

---

## 2. Architecture and Components

LangChain applications follow a modular architecture with well-defined component responsibilities.

### System Architecture

![LangChain Architecture](images/02_demo_app/langchain_architecture.png)

### Component Deep Dive

#### Prompt Templates

**Purpose**: Dynamic prompt construction with variable interpolation

**Implementation**: `ChatPromptTemplate` for multi-turn conversations

**Key Features**:
- Variable substitution with `{variable}` syntax
- Message role specification (system, user, assistant)
- Template validation and type checking

#### Language Models

**Purpose**: Text generation and completion

**Implementation**: `ChatOpenAI` wrapper for OpenAI's API

**Configuration Options**:
- Model selection (gpt-4, gpt-3.5-turbo, etc.)
- Temperature control for randomness
- Token limits and streaming options

#### Output Parsers

**Purpose**: Response formatting and structure extraction

**Implementation**: `StrOutputParser` for string extraction

**Alternative Parsers**:
- `PydanticOutputParser` for structured data
- `JSONOutputParser` for JSON responses
- `CommaSeparatedListOutputParser` for lists

### Data Flow Analysis

![LangChain Data Flow](images/02_demo_app/langchain_data_flow.png)

The data flow demonstrates the four-step process from user input to final output:

1. **Template Variable Injection**: Input parameters are injected into prompt template
2. **Prompt Formatting**: System and user messages are properly formatted
3. **LLM Processing**: OpenAI API processes the prompt and returns AIMessage
4. **Output Parsing**: AIMessage content is extracted as clean string

---

## 3. Implementation

---

### 3.1 Environment Setup

Import required LangChain components and configure environment variables for API access.

**Dependencies**:
- `langchain-openai`: OpenAI model integrations
- `langchain-core`: Core abstractions and utilities
- `python-dotenv`: Environment variable management

**Security**: API keys loaded from `.env` file to avoid hardcoding sensitive information.

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from IPython.display import Markdown, display
import os

# Load environment variables from .env file
_ = load_dotenv()

# Verify API key is available
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY environment variable"

print("LangChain components imported successfully")
print("Environment variables loaded")

LangChain components imported successfully
Environment variables loaded


### 3.2 Model Configuration

Initialize the ChatOpenAI model with specific configuration for consistent, high-quality outputs.

**Configuration Parameters**:
- `model_name`: GPT-4 for high-quality reasoning and planning
- `temperature=0`: Deterministic outputs for reproducible results
- Default streaming and async support enabled automatically

**Model Selection Rationale**:
- GPT-4: Superior reasoning capabilities for complex trip planning
- Temperature 0: Ensures consistent recommendations for same inputs
- Cost consideration: GPT-4 more expensive but provides better results

In [24]:
# Initialize ChatOpenAI with GPT-4
model = ChatOpenAI(
    model_name="gpt-4",  # High-quality model for complex reasoning
    temperature=0        # Deterministic outputs
)

print(f"Model configured: {model.model_name} with temperature={model.temperature}")
print(f"Streaming enabled: {model.streaming}")

Model configured: gpt-4 with temperature=0.0
Streaming enabled: False


### 3.3 Prompt Template Design

Create a structured prompt template using ChatPromptTemplate for multi-turn conversation management.

**Template Structure**:
- **System Message**: Defines agent role and behavior constraints
- **User Message**: Contains the actual query with variable substitution

**Variable Substitution**:
- `{destination}`: Target location for trip planning
- `{preferences}`: User preferences (activities, budget, style)

**Design Principles**:
- Clear role definition ("trip planner expert")
- Specific instruction for preference consideration
- Flexible variable system for different use cases

**Prompt Engineering Best Practices Applied**:
- Explicit role assignment for consistent behavior
- Context injection through variables
- Clear task specification

In [25]:
# System prompt defining agent behavior
system_template = """
You are a trip planner expert. Help me plan a trip to {destination}.
Consider my preferences for {preferences}.
Provide specific, actionable recommendations with practical details.
"""

# Create ChatPromptTemplate with system and user messages
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', 'What should I do in {destination}? Please provide detailed recommendations.')
])

print(f"Prompt template created with {len(prompt_template.messages)} messages:")
print("1. System message with role definition")
print("2. User message with query template")
print(f"\nVariables used: {prompt_template.input_variables}")

Prompt template created with 2 messages:
1. System message with role definition
2. User message with query template

Variables used: ['destination', 'preferences']


### 3.4 Output Parser Configuration

Configure output parsing to extract clean string responses from model outputs.

**StrOutputParser Features**:
- Extracts `.content` field from AIMessage objects
- Returns clean string without metadata
- Compatible with all text-based LLM outputs
- No additional configuration required

**Alternative Parsers for Different Use Cases**:

| Parser Type | Use Case | Output Format |
|-------------|----------|---------------|
| `StrOutputParser` | Simple text responses | `str` |
| `PydanticOutputParser` | Structured data | Custom Pydantic models |
| `JSONOutputParser` | JSON responses | `dict` |
| `CommaSeparatedListOutputParser` | List outputs | `list[str]` |
| `DatetimeOutputParser` | Date/time parsing | `datetime` |

**Implementation Note**: StrOutputParser selected for simplicity and compatibility with trip planning text outputs.

In [26]:
# Initialize string output parser
parser = StrOutputParser()

print(f"Output parser configured: {parser.__class__.__name__}")
print("Purpose: Extract string content from AIMessage objects")

Output parser configured: StrOutputParser
Purpose: Extract string content from AIMessage objects


### 3.5 LCEL Chain Composition

Compose the complete processing chain using LCEL pipe operator syntax.

**Chain Definition**:
```python
chain = prompt_template | model | parser
```

**Execution Flow**:
1. **Input** → `prompt_template`: Variable substitution and message formatting
2. **Formatted Prompt** → `model`: LLM processing and response generation
3. **AIMessage** → `parser`: Content extraction and string conversion
4. **Output** → Clean string response

**LCEL Benefits in This Chain**:
- **Automatic Type Checking**: Input/output compatibility validation
- **Streaming Support**: Real-time response generation capability
- **Error Propagation**: Automatic error handling across components
- **Observability**: Built-in logging and tracing

**Chain Properties**:
- **Stateless**: No internal state between invocations
- **Reusable**: Can be called multiple times with different inputs
- **Composable**: Can be used as component in larger chains
- **Serializable**: Can be saved and loaded for deployment

In [27]:
# Compose the complete chain using LCEL
trip_planner_chain = prompt_template | model | parser

print("Trip planner chain created successfully")
print("Chain components: 3")
print("  1. ChatPromptTemplate (prompt formatting)")
print("  2. ChatOpenAI (LLM processing)")
print("  3. StrOutputParser (output extraction)")
print()
print("Chain capabilities:")
print("  ✓ Synchronous execution")
print("  ✓ Asynchronous execution")
print("  ✓ Streaming support")
print("  ✓ Batch processing")

Trip planner chain created successfully
Chain components: 3
  1. ChatPromptTemplate (prompt formatting)
  2. ChatOpenAI (LLM processing)
  3. StrOutputParser (output extraction)

Chain capabilities:
  ✓ Synchronous execution
  ✓ Asynchronous execution
  ✓ Streaming support
  ✓ Batch processing


## 4. Trip Planner Application

---

### 4.1 Function Interface Design

Create a clean, reusable function interface for the trip planning chain.

**Design Principles**:
- **Type Hints**: Clear input/output type specification
- **Documentation**: Comprehensive docstring with parameters and return values
- **Error Handling**: Graceful handling of API failures
- **Input Validation**: Parameter validation before chain execution

**Function Signature**:
```python
def plan_trip(destination: str, preferences: str) -> str
```

**Implementation Strategy**:
- Dictionary-based input for chain compatibility
- Synchronous execution for simplicity
- String return type for easy integration

In [28]:
def plan_trip(destination: str, preferences: str) -> str:
    """
    Generate trip recommendations for a destination based on user preferences.

    Args:
        destination: Target location for the trip (e.g., "Paris", "Tokyo")
        preferences: User preferences and requirements (e.g., "museums, food, budget-friendly")

    Returns:
        str: Detailed trip recommendations and suggestions
    """
    # Input validation
    if not destination or not destination.strip():
        raise ValueError("Destination cannot be empty")
    if not preferences or not preferences.strip():
        raise ValueError("Preferences cannot be empty")

    try:
        # Prepare input data for chain
        input_data = {
            "destination": destination.strip(),
            "preferences": preferences.strip()
        }

        # Execute the chain
        result = trip_planner_chain.invoke(input_data)
        return result

    except Exception as e:
        # Log error and re-raise with context
        error_msg = f"Trip planning failed for {destination}: {str(e)}"
        raise Exception(error_msg) from e

def render_markdown(content: str):
    """
    Render the given content as Markdown in the notebook.

    Args:
        content: The string content to render as Markdown
    """
    display(Markdown(content))

print("Trip planning function defined")
print("Function signature: plan_trip(destination: str, preferences: str) -> str")
print("Helper function: render_markdown() for enhanced output display")

Trip planning function defined
Function signature: plan_trip(destination: str, preferences: str) -> str
Helper function: render_markdown() for enhanced output display


### 4.2 Practical Examples

Demonstrate the trip planner with various real-world scenarios.

---

#### 4.2.1 Paris Cultural Trip

Plan a cultural trip to Paris focusing on art and cuisine.

In [29]:
# Example 1: Cultural trip to Paris
destination_1 = "Paris"
preferences_1 = "art museums, French cuisine, cultural experiences"

print(f"TRIP PLAN: {destination_1} (Art Museums & French Cuisine)")
print("=" * 60)

result_1 = plan_trip(destination_1, preferences_1)
render_markdown(result_1)
print("=" * 60)

TRIP PLAN: Paris (Art Museums & French Cuisine)


Sure, I'd be happy to help you plan your trip to Paris. Given your interests in art museums, French cuisine, and cultural experiences, here's a detailed itinerary:

**Day 1: Arrival and Eiffel Tower**

Upon your arrival in Paris, settle into your hotel and then head to the iconic Eiffel Tower. You can take an elevator ride up to the top for a stunning view of the city. Afterwards, enjoy a dinner at "Le Jules Verne" restaurant located on the second floor of the Eiffel Tower.

**Day 2: Louvre Museum and French Cuisine**

Start your day with a visit to the Louvre Museum. It's the world's largest art museum and a historic monument in Paris. It's home to thousands of works of art, including the Mona Lisa and the Venus de Milo. I recommend booking a skip-the-line ticket in advance to avoid long queues.

For lunch, head to "Le Comptoir du Relais" in the Saint-Germain district. This bistro offers traditional French cuisine in a cozy setting.

In the evening, take a stroll along the Seine River and enjoy the beautiful city lights.

**Day 3: Montmartre and Sacré-Cœur**

Spend your third day exploring the bohemian district of Montmartre. Visit the Sacré-Cœur Basilica, enjoy the panoramic view of Paris from its highest point, and stroll around the Place du Tertre, a square known for its artists and portraitists.

For dinner, try "Le Consulat", a historic restaurant that offers delicious French cuisine. It's also a great place to try escargot, a classic French dish.

**Day 4: Musée d'Orsay and French Wine Tasting**

Visit the Musée d'Orsay, which houses French art dating from 1848 to 1914, including works by Monet, Degas, Renoir, and Van Gogh. 

In the afternoon, join a French wine tasting tour. "O Chateau" offers a variety of wine tasting options, from beginner to expert level, and it's located in the center of Paris.

**Day 5: Versailles Palace**

Take a day trip to the Palace of Versailles. It's one of the most famous castles in the world and a UNESCO World Heritage site. You can explore the grand palace, the beautiful gardens, and the Hall of Mirrors. 

For dinner, "Gordon Ramsay au Trianon" is a two-Michelin-star restaurant located in Versailles, offering French cuisine with a British twist.

**Day 6: Shopping and Farewell Dinner**

Spend your last day in Paris shopping. The Champs-Élysées is a famous avenue where you can find luxury boutiques. 

For your farewell dinner, "L'Ambroisie" is a three-Michelin-star restaurant located in the heart of Paris on the Place des Vosges. It offers classic French cuisine in an elegant setting.

**Day 7: Departure**

Before your departure, have a relaxing breakfast at your hotel and make sure you have all your belongings. Arrive at the airport in good time for your flight.

Remember to check the opening days and hours of the places you plan to visit, as some museums and restaurants close on certain days of the week. Enjoy your trip to Paris!

#### 4.2.2 Tokyo Technology and Food Adventure

Explore Tokyo's technology scene and culinary diversity.

In [30]:
# Example 2: Technology and food adventure in Tokyo
destination_2 = "Tokyo"
preferences_2 = "technology, innovation, authentic Japanese food, modern experiences"

print(f"TRIP PLAN: {destination_2} (Technology & Food)")
print("=" * 60)

result_2 = plan_trip(destination_2, preferences_2)
render_markdown(result_2)
print("\n" + "=" * 60)

TRIP PLAN: Tokyo (Technology & Food)


Sure, I'd be happy to help you plan your trip to Tokyo. Here's a detailed itinerary considering your preferences for technology, innovation, authentic Japanese food, and modern experiences.

Day 1: Arrival and Explore Akihabara
- Arrive in Tokyo and check into your hotel.
- Head to Akihabara, known as the center of Japan's otaku (diehard fan) culture. Here you can find many electronics shops, anime and manga stores.
- Visit the Akihabara Radio Kaikan for a variety of anime and manga related goods.
- Explore the Yodobashi Camera, a large electronics store where you can find the latest Japanese tech products.
- Have dinner at Izakaya, a type of informal Japanese pub. They are casual places for after-work drinking and serving a variety of dishes.

Day 2: Odaiba and TeamLab Borderless
- Start your day by visiting Odaiba, a popular shopping and entertainment district on a man-made island in Tokyo Bay.
- Visit the teamLab Borderless, a group of ultra-technologists whose collaborative, immersive digital art exhibitions stimulate all five senses. Pre-book your tickets as they often sell out.
- Have lunch at Aqua City, a large shopping mall featuring various Japanese and international restaurants.
- In the evening, visit the Odaiba Seaside Park and enjoy the beautiful view of Rainbow Bridge and Tokyo Bay.
- For dinner, try out the local cuisine at one of the restaurants in DiverCity Tokyo Plaza.

Day 3: Tsukiji Fish Market and Ginza
- Start your day early with a visit to the Tsukiji Fish Market. Try the freshest sushi and sashimi for breakfast.
- Take a sushi-making class to learn about the art of sushi making from a professional sushi chef.
- Head to Ginza, Tokyo's most famous upmarket shopping, dining, and entertainment district.
- Visit the Sony Park, a unique, constantly evolving park with a mix of indoor and outdoor spaces. It's a great place to experience the latest technology and design concepts from Sony.
- Have dinner at Sukiyabashi Jiro, a renowned sushi restaurant in Ginza. Remember to make a reservation in advance.

Day 4: Shibuya and Harajuku
- Start your day in Shibuya. Visit the Shibuya Crossing, one of the world's most heavily used pedestrian scrambles.
- Visit the Hachiko statue, a famous meeting point and landmark in Shibuya.
- Head to Harajuku, the center of Japan's most extreme teenage cultures and fashion styles.
- Visit the Meiji Shrine, located in a forest that covers an area of 70 hectares.
- Explore Takeshita Street, packed with shops selling clothes, quirky fashion accessories, and crepe stands.
- Have dinner at Uobei Shibuya Dogenzaka, a high-tech sushi restaurant where you can order on a touchscreen and have your food delivered by a high-speed chute.

Day 5: Departure
- Depending on your flight time, you might want to explore more of the city or do some last-minute shopping.
- Head to the airport for your departure.

Remember to check the opening times and days for all the places you plan to visit. Also, consider getting a Suica card for convenient travel on public transportation in Tokyo. Enjoy your trip!

#### 4.2.3 Budget Backpacking in Southeast Asia

Plan an economical adventure through Thailand.

In [31]:
# Example 3: Budget backpacking adventure
destination_3 = "Thailand"
preferences_3 = "budget backpacking, adventure activities, cultural experiences, low cost"

print(f"TRIP PLAN: {destination_3} (Budget Backpacking)")
print("=" * 60)

result_3 = plan_trip(destination_3, preferences_3)
render_markdown(result_3)  # Fixed: was result_1, now correct result_3
print("=" * 60)

TRIP PLAN: Thailand (Budget Backpacking)


Sure, here's a detailed plan for your budget backpacking trip to Thailand:

1. **Bangkok**: Start your journey in the capital city. Visit the Grand Palace, Wat Arun, and Wat Phra Kaew. Explore the bustling street markets like Chatuchak Weekend Market and Chinatown. Try street food, especially Pad Thai, Mango Sticky Rice, and Tom Yum Soup. Stay in budget hostels like Lub d Bangkok Silom or Nappark Hostel near Khao San Road. 

2. **Ayutthaya**: Take a train from Bangkok to Ayutthaya, a UNESCO World Heritage Site. Explore the historical ruins of temples and statues. You can rent a bike for a day to tour around the city. 

3. **Chiang Mai**: Fly or take a bus to Chiang Mai. Visit the old city, Doi Suthep Temple, and enjoy the night bazaar. Participate in a traditional Thai cooking class. Stay in budget hostels like Stamps Backpackers or Hug Hostel. 

4. **Pai**: Take a minivan from Chiang Mai to Pai, a small town in the mountains known for its natural beauty. Visit the hot springs, Pai Canyon, and waterfalls. Rent a scooter to get around. 

5. **Krabi**: Fly to Krabi from Chiang Mai. Visit Railay Beach, go rock climbing, and explore the Emerald Pool. Take a boat tour to the Four Islands. Stay in Slumber Party Hostel for a budget-friendly option.

6. **Koh Phi Phi**: Take a ferry from Krabi. Enjoy the beach, go snorkeling or scuba diving. Hike to the viewpoint for a stunning view of the island. 

7. **Phuket**: End your trip in Phuket. Visit Patong Beach, Old Phuket Town, and Big Buddha. Enjoy the nightlife in Bangla Road. 

**Adventure Activities**: Rock climbing in Railay, trekking in Chiang Mai, snorkeling and scuba diving in Koh Phi Phi, and exploring the caves in Pai.

**Cultural Experiences**: Visit temples in Bangkok and Chiang Mai, explore the historical ruins in Ayutthaya, participate in a Thai cooking class in Chiang Mai, and experience the local markets in Bangkok.

**Budget Tips**: Use local transport like buses, trains, and ferries. Eat at local markets and street food stalls. Stay in budget hostels and guesthouses. Book activities and tours directly from local operators to get a better price.

Remember to respect the local customs and traditions. Dress modestly when visiting temples. Always ask for permission before taking photos of people. Enjoy your trip!

### 4.3 Input Validation and Error Handling

Demonstrate the function's error handling capabilities.

In [32]:
print("ERROR HANDLING DEMONSTRATIONS:")
print("=" * 50)

# Test 1: Empty destination
try:
    plan_trip("", "cultural experiences")
except ValueError as e:
    print(f"\nTest 1: Empty destination")
    print(f"✓ Caught expected error: {e}")

# Test 2: Empty preferences
try:
    plan_trip("Paris", "")
except ValueError as e:
    print(f"\nTest 2: Empty preferences")
    print(f"✓ Caught expected error: {e}")

# Test 3: Whitespace only
try:
    plan_trip("   ", "museums")
except ValueError as e:
    print(f"\nTest 3: Whitespace-only inputs")
    print(f"✓ Caught expected error: {e}")

print("\nAll error handling tests passed!")

ERROR HANDLING DEMONSTRATIONS:

Test 1: Empty destination
✓ Caught expected error: Destination cannot be empty

Test 2: Empty preferences
✓ Caught expected error: Preferences cannot be empty

Test 3: Whitespace-only inputs
✓ Caught expected error: Destination cannot be empty

All error handling tests passed!


## 5. Advanced Features

Explore advanced LCEL capabilities for enhanced functionality.

---

### 5.1 Streaming Support

Implement real-time response streaming for better user experience.

**Streaming Benefits**:
- **Reduced perceived latency**: Users see responses immediately
- **Better UX**: Progressive content loading
- **Chunked processing**: Handle long responses efficiently
- **Early termination**: Stop generation if needed

**LCEL Streaming**: Automatic support through chain composition.

In [33]:
def plan_trip_streaming(destination: str, preferences: str):
    """
    Stream trip recommendations in real-time for better user experience.

    Args:
        destination: Target location
        preferences: User preferences

    Yields:
        str: Streaming chunks of the response
    """
    input_data = {"destination": destination, "preferences": preferences}

    # Stream the response
    for chunk in trip_planner_chain.stream(input_data):
        yield chunk

# Demonstrate streaming
print("STREAMING DEMO: Rome trip planning")
print("=" * 50)
print()

# Stream Rome trip planning response
for chunk in plan_trip_streaming("Rome", "ancient history, Italian cuisine"):
    print(chunk, end="", flush=True)

print("\n\nSTREAMING COMPLETE")
print("=" * 50)

STREAMING DEMO: Rome trip planning

Sure, I'd be happy to help you plan your trip to Rome. Given your interest in ancient history and Italian cuisine, here are some recommendations:

Day 1: Arrival and Exploring the City
- Arrive in Rome and check into your hotel. Depending on your arrival time, you may want to take a leisurely stroll around your hotel area to get a feel for the city.
- Have dinner at a local restaurant. I recommend "Trattoria Vecchia Roma" known for its traditional Roman dishes.

Day 2: Ancient Rome
- Start your day with a visit to the Colosseum, one of the most iconic symbols of Rome. It's best to book a guided tour in advance to avoid long queues and to get a deeper understanding of its history.
- After the Colosseum, head to the Roman Forum and Palatine Hill, which are in the same archaeological area.
- For lunch, try "La Taverna dei Fori Imperiali", a family-run restaurant near the Roman Forum.
- In the afternoon, visit the Pantheon, a former Roman temple that's n

### 5.2 Batch Processing

Process multiple trip requests simultaneously for efficiency.

**Batch Benefits**:
- **Performance**: Parallel API calls
- **Cost efficiency**: Reduced overhead
- **Throughput**: Handle multiple requests
- **Resource optimization**: Better API utilization

In [34]:
def plan_trips_batch(requests: list[dict]) -> list[str]:
    """
    Process multiple trip planning requests in batch for efficiency.

    Args:
        requests: List of {"destination": str, "preferences": str}

    Returns:
        list[str]: Trip plans corresponding to each request
    """
    return trip_planner_chain.batch(requests)

# Demonstrate batch processing
print("BATCH PROCESSING DEMO")
print("=" * 50)

# Prepare batch requests
batch_requests = [
    {
        "destination": "London",
        "preferences": "museums, theater, British culture"
    },
    {
        "destination": "Bali",
        "preferences": "wellness, nature, spiritual experiences"
    },
    {
        "destination": "New York",
        "preferences": "art galleries, diverse food, urban exploration"
    }
]

print(f"\nProcessing {len(batch_requests)} trip requests simultaneously...\n")

# Execute batch processing
batch_results = plan_trips_batch(batch_requests)

print("BATCH RESULTS:")
print("=" * 50)

# Display batch results summary
for i, (request, result) in enumerate(zip(batch_requests, batch_results), 1):
    destination = request["destination"]
    preferences = request["preferences"].split(",")[0].title() + " & " + request["preferences"].split(",")[1].title()

    print(f"\n{i}. {destination.upper()} ({preferences})")
    print(f"   Result length: {len(result):,} characters")
    print(f"   ✓ Plan generated successfully")

print(f"\nAll {len(batch_requests)} requests completed successfully!")
print("Total processing time optimized through batch execution.")

BATCH PROCESSING DEMO

Processing 3 trip requests simultaneously...

BATCH RESULTS:

1. LONDON (Museums &  Theater)
   Result length: 2,443 characters
   ✓ Plan generated successfully

2. BALI (Wellness &  Nature)
   Result length: 2,542 characters
   ✓ Plan generated successfully

3. NEW YORK (Art Galleries &  Diverse Food)
   Result length: 2,777 characters
   ✓ Plan generated successfully

All 3 requests completed successfully!
Total processing time optimized through batch execution.


### 5.3 Asynchronous Processing

Implement async support for non-blocking operations.

**Async Benefits**:
- **Concurrency**: Handle multiple requests simultaneously
- **Scalability**: Better resource utilization
- **Responsiveness**: Non-blocking I/O operations
- **Integration**: Compatible with async web frameworks

In [35]:
import asyncio

async def plan_trip_async(destination: str, preferences: str) -> str:
    """
    Asynchronously plan a trip for non-blocking execution.

    Args:
        destination: Target location
        preferences: User preferences

    Returns:
        str: Trip recommendations
    """
    input_data = {"destination": destination, "preferences": preferences}
    result = await trip_planner_chain.ainvoke(input_data)
    return result

async def async_demo():
    """Demonstrate async trip planning."""
    print("ASYNC PROCESSING DEMO")
    print("=" * 50)
    print("\nStarting async trip planning...")

    # Execute async trip planning
    result = await plan_trip_async("Berlin", "history, nightlife, alternative culture")

    print(f"✓ Berlin trip plan completed ({len(result):,} characters)")
    print("\nASYNC PROCESSING SUCCESSFUL")
    print("Async execution enables non-blocking operations and better scalability.")

# Run async demo
await async_demo()

ASYNC PROCESSING DEMO

Starting async trip planning...
✓ Berlin trip plan completed (2,127 characters)

ASYNC PROCESSING SUCCESSFUL
Async execution enables non-blocking operations and better scalability.


## 6. References

**Official Documentation**:
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [LCEL (LangChain Expression Language)](https://python.langchain.com/docs/concepts/lcel)
- [OpenAI API Documentation](https://platform.openai.com/docs/api-reference)
- [ChatOpenAI Integration](https://python.langchain.com/docs/integrations/chat/openai)

**Core Concepts**:
- [Prompt Templates](https://python.langchain.com/docs/concepts/prompt_templates)
- [Output Parsers](https://python.langchain.com/docs/concepts/output_parsers)
- [Runnable Interface](https://python.langchain.com/docs/concepts/runnables)
- [Streaming in LangChain](https://python.langchain.com/docs/concepts/streaming)

**Advanced Features**:
- [LangChain Agents](https://python.langchain.com/docs/concepts/agents)
- [Memory Systems](https://python.langchain.com/docs/concepts/memory)
- [Vector Stores](https://python.langchain.com/docs/concepts/vectorstores)
- [Callbacks and Monitoring](https://python.langchain.com/docs/concepts/callbacks)

**Production Deployment**:
- [LangServe](https://python.langchain.com/docs/langserve) - REST API deployment
- [LangSmith](https://smith.langchain.com/) - Monitoring and evaluation
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/production-best-practices)

**Related Papers**:
- [Chain-of-Thought Prompting](https://arxiv.org/abs/2201.11903)
- [Language Models as Tool Users](https://arxiv.org/abs/2302.04761)
- [ReAct: Synergizing Reasoning and Acting](https://arxiv.org/abs/2210.03629)

**Community Resources**:
- [LangChain GitHub Repository](https://github.com/langchain-ai/langchain)
- [LangChain Community](https://github.com/langchain-ai/langchain-community)
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)

---

*Last updated: October 2025*